In [4]:
import pandas as pd
import numpy as np
np.random.seed(2)

In [11]:
df = pd.DataFrame()
user_id = np.random.randint(1,501,10000)
df['user_id'] = user_id
date = pd.date_range(start='20190101',periods=365)
df['trans_date'] = np.random.choice(date,size=10000)
money = np.random.randint(1,501,10000)
df['money'] = money

处理R值

In [40]:
def format_R(x):
    if x<15:
        return 5
    elif x<30:
        return 4
    elif x<60:
        return 3
    elif x<90:
        return 2
    else:
        return 1
    
df_r = df.groupby('user_id',as_index=False).max()
df_r['days'] = df_r.trans_date.apply(lambda x:(pd.to_datetime('2020-01-01')-x).days)
df_r['R'] = df_r.days.apply(format_R)
df_r.head()


,user_id,trans_date,money,days,R
0,1,2019-12-03,462,29,4
1,2,2019-12-26,493,6,5
2,3,2019-12-26,478,6,5
3,4,2019-12-08,495,24,4
4,5,2019-12-25,471,7,5


处理F值

In [42]:
def format_F(x):
    if x<8:
        return 1
    elif x<16:
        return 2
    elif x<24:
        return 3
    else:
        return 4

df_f = df.groupby('user_id',as_index=False).count()
df_f['F'] = df_f.money.apply(format_F)
df_f.head()

,user_id,trans_date,money,F
0,1,15,15,2
1,2,22,22,3
2,3,13,13,2
3,4,17,17,3
4,5,23,23,3


处理M值

In [46]:
def format_M(x):
    if x<3400:
        return 1
    elif x<5400:
        return 2
    elif x<7400:
        return 3
    else:
        return 4

df_m = df.groupby('user_id',as_index=False)['money'].sum()
df_m['M'] = df_m.money.apply(format_M)
df_m.head()

,user_id,money,M
0,1,4167,2
1,2,4528,2
2,3,4006,2
3,4,3652,2
4,5,5820,3


In [69]:
rfm_model = pd.merge(pd.merge(df_r,df_f,on='user_id'),df_m,on='user_id')
rfm_model=rfm_model[['user_id','R','F','M']].set_index('user_id')
rfm_model.head()

,R,F,M
user_id,,,
1,4,2,2
2,5,3,2
3,5,2,2
4,4,3,2
5,5,3,3


In [70]:
rfm_model = rfm_model/rfm_model.mean()
rfm_model.head()

,R,F,M
user_id,,,
1,0.914495,0.650195,0.865801
2,1.143118,0.975293,0.865801
3,1.143118,0.650195,0.865801
4,0.914495,0.975293,0.865801
5,1.143118,0.975293,1.298701


In [71]:
def format_rfm(x):
    if x<1:
        return '↓'
    else:
        return '↑'
rfm_model = rfm_model.applymap(format_rfm)
rfm_model.head()

,R,F,M
user_id,,,
1,↓,↓,↓
2,↑,↓,↓
3,↑,↓,↓
4,↓,↓,↓
5,↑,↓,↑
